# Investigating Potentials

In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [83]:
import sympy as sy
# Import all available functions (bad practice but ok for our example)
from sympy.functions import *
from ipywidgets import HBox, IntSlider, FloatSlider, VBox, Text, Layout
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit, minimize

We want to have a look at $F = -fx + \frac{a}{x}$, find the potential and 

In [50]:
x, a, f = sy.symbols('x a f')

F = -f * x + a/x**3
U = -sy.integrate(F, x)
U_np = sy.lambdify([x, a, f], U)

In [6]:
def pot_plot(F, name):
    a_val = FloatSlider(value=1, min=0, max=5, description='a: ')
    f_val = FloatSlider(value=1, min=0, max=5, description='f: ')
    
    fig, ax = figure(name)
    U_np = sy.lambdify([x, a, f], U) 
    
    xs = np.linspace(0, 5, 101)
    line, = ax.plot(xs, U_np(xs, 1, 1))
    
    ax.set_ylim([-0.1, 13.1])
    
    def update_a(change):
        redraw(change.new, f_val.value)

    def update_f(change):
        redraw(a_val.value, change.new)

    def redraw(a0, f0):
        line.set_data(xs, U_np(xs, a0, f0))
        fig.canvas.draw()
        fig.canvas.flush_events()    
        
    a_val.observe(update_a, names='value')
    f_val.observe(update_f, names='value')
    return HBox([a_val, f_val])

pot_plot(F, 'potential')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<string>:2: RuntimeWarning: divide by zero encountered in true_divide


In [112]:
F_np = sy.lambdify([x, a, f], F) 

def find_start_pos(U, E, a0, f0):
    xs = np.linspace(1e-10, 3, 10001)
    Fs = sy.lambdify([x, a, f], F)(xs, a0, f0)
    x_min = xs[abs(Fs).argmin()]
    xs = np.linspace(1e-10, x_min, 10001)
    Us = U_np(xs, a0, f0)
    x1 = xs[(np.abs(Us - E)).argmin()]
    xs = np.linspace(x_min, 5, 10001)
    Us = U_np(xs, a0, f0)
    x2 = xs[(np.abs(Us - E)).argmin()]
    return [x1, x_min, x2]
    
def derivative(t, x, a0, f0):
    return [
        x[1],
        F_np(x[0], a0, f0)
    ]

def func1(x, a, b, c):
    return - a * np.cos(b*x) + c

def func2(x, a, b, c):
    return a * np.abs(np.cos(b*x + np.pi/2)) + c

def fit_func(func, xdata, ydata, p0=[1, 1, 1]):
    popt, pcov = curve_fit(func, xdata, ydata, p0=p0)
    return func(xdata, *popt)

def min_fun(args, xdata, ydata, func):
    return np.sum((ydata - func(xdata, *args))**2)

def fit_func2(func, xdata, ydata, p0=[1, 1, 1]):
    res = minimize(min_fun, p0, args=(xdata, ydata, func), method='BFGS',
                  options={'disp':False})
    return func(xdata, *res.x), res.fun

In [120]:
def dyn_plot(F, name):
    a_val = FloatSlider(value=1, min=0, max=5, description='a: ')
    f_val = FloatSlider(value=1, min=0, max=5, description='f: ')
    E_val = FloatSlider(value=3, min=.1, max=10, description='E: ')

    fig, ax = figure(name, 1, 2, figsize=(12, 4))
    U_np = sy.lambdify([x, a, f], U) 
    
    xs = np.linspace(0, 5, 101)
    
    X0 = [find_start_pos(U, E_val.value, a_val.value, f_val.value)[0], 0]
    times = np.linspace(0, 10, 300)
    sol = solve_ivp(derivative, [times[0], times[-1]], X0, t_eval=times, args=(a_val.value, f_val.value))
    
    fit1, res1 = fit_func2(func1, sol.t, sol.y[0], p0=[1, 2, 0.9])
    fit2, res2 = fit_func2(func2, sol.t, sol.y[0], p0=[1, 1, 0.9])
                
    lines = [ax[0].plot(xs, [E_val.value]*len(xs), 'k--')[0],
             ax[0].plot(xs, U_np(xs, 1, 1))[0],
             ax[1].plot(sol.t, fit1, 'k--')[0],
             ax[1].plot(sol.t, fit2, 'k')[0],
             ax[1].plot(sol.t, sol.y[0], 'r',  lw=2)[0],]
    
    
    
    ax[0].set_ylim([-0.1, 13.1])
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('U(x)')
    ax[1].set_xlabel('t')
    ax[1].set_ylabel('x(t)')
    
    
    def update_a(change):
        redraw(change.new, f_val.value, E_val.value)

    def update_f(change):
        redraw(a_val.value, change.new, E_val.value)

    def update_E(change):
        redraw(a_val.value, f_val.value, change.new)
        
    def redraw(a0, f0, E0):
        
        lines[0].set_data(xs, [E0]*len(xs))
        lines[1].set_data(xs, U_np(xs, a0, f0))
        
        x0 = find_start_pos(U, E0, a_val.value, f_val.value)
        if U_np(x0[1], a_val.value, f_val.value) > E_val.value:
            E_val.value = U_np(x0[1], a_val.value, f_val.value)            
        X0 = [x0[0], 0]
        
        sol = solve_ivp(derivative, [times[0], times[-1]], X0, t_eval=times, args=(a_val.value, f_val.value))
        lines[4].set_data(sol.t, sol.y[0])
        
        fit1, res1 = fit_func2(func1, sol.t, sol.y[0], p0=[x0[2] - x0[0], 2*np.sqrt(f_val.value), x0[1]])
        fit2, res2 = fit_func2(func2, sol.t, sol.y[0], p0=[x0[2] - x0[0], np.sqrt(f_val.value), x0[1]])
        
        lines[2].set_data(sol.t, fit1)
        lines[3].set_data(sol.t, fit2)
        
        inds = [2, 3] if res1 > res2 else [3, 2]
        lines[inds[0]].set_linestyle('dashed')
        lines[inds[1]].set_linestyle('solid')
        
        ax[1].relim()
        # update ax.viewLim using the new dataLim
        ax[1].autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()    
        
    a_val.observe(update_a, names='value')
    f_val.observe(update_f, names='value')
    E_val.observe(update_E, names='value')
    return HBox([a_val, f_val, E_val])

dyn_plot(F, 'dyn')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<string>:2: RuntimeWarning: divide by zero encountered in true_divide
